<a href="https://colab.research.google.com/github/GeulNoon/GeulNoon/blob/Start/Kobart%20%EC%9A%94%EC%95%BD%EB%AC%B8%2C%20%EC%82%AC%EC%9A%A9%EC%9E%90%20%EC%9A%94%EC%95%BD%EB%AC%B8%20%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentence Transformers 학습

In [1]:
pip install sentence-transformers datasets

     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 298 kB 24.9 MB/s 
     |████████████████████████████████| 3.1 MB 41.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 1.2 MB 44.4 MB/s 
     |████████████████████████████████| 61 kB 534 kB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 243 kB 27.2 MB/s 
     |████████████████████████████████| 132 kB 53.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.1 MB/s 
     |████████████████████████████████| 271 kB 49.7 MB/s 
     |████████████████████████████████| 160 kB 50.7 MB/s 
     |████████████████████████████████| 192 kB 51.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=43b490c8d93fe23281c1747a23558ae9b25a0b1eece86fa4fdfd398c5711e4af
  Stored in

In [2]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [4]:
model_name = "klue/roberta-base"

In [5]:
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [6]:
embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [8]:
model = SentenceTransformer(modules=[embedding_model, pooler])

2021-12-08 17:17:15 - Use pytorch device: cuda


In [9]:
datasets = load_dataset("klue", "sts")

Downloading:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

In [11]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [12]:
testsets = load_dataset("kor_nlu", "sts")

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.1k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset kor_nlu downloaded and prepared to /root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
testsets

DatasetDict({
    train: Dataset({
        features: ['genre', 'filename', 'year', 'id', 'score', 'sentence1', 'sentence2'],
        num_rows: 5703
    })
    validation: Dataset({
        features: ['genre', 'filename', 'year', 'id', 'score', 'sentence1', 'sentence2'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['genre', 'filename', 'year', 'id', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})

In [14]:
testsets["test"][0]

{'filename': 2,
 'genre': 1,
 'id': 24,
 'score': 2.5,
 'sentence1': '한 소녀가 머리를 스타일링하고 있다.',
 'sentence2': '한 소녀가 머리를 빗고 있다.',
 'year': 6}

In [15]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [16]:
train_samples[0].texts, train_samples[0].label

(['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
  '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'],
 0.74)

In [17]:
test_samples[0].texts, test_samples[0].label

(['한 소녀가 머리를 스타일링하고 있다.', '한 소녀가 머리를 빗고 있다.'], 0.5)

In [18]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [19]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [20]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2021-12-08 17:17:29 - Warmup-steps: 146


In [21]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 17:23:30 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-12-08 17:23:34 - Cosine-Similarity :	Pearson: 0.8682	Spearman: 0.8640
2021-12-08 17:23:34 - Manhattan-Distance:	Pearson: 0.8682	Spearman: 0.8612
2021-12-08 17:23:34 - Euclidean-Distance:	Pearson: 0.8686	Spearman: 0.8616
2021-12-08 17:23:34 - Dot-Product-Similarity:	Pearson: 0.8574	Spearman: 0.8494
2021-12-08 17:23:34 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_17-16-47


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 17:29:27 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2021-12-08 17:29:31 - Cosine-Similarity :	Pearson: 0.8825	Spearman: 0.8792
2021-12-08 17:29:31 - Manhattan-Distance:	Pearson: 0.8805	Spearman: 0.8749
2021-12-08 17:29:31 - Euclidean-Distance:	Pearson: 0.8814	Spearman: 0.8763
2021-12-08 17:29:31 - Dot-Product-Similarity:	Pearson: 0.8726	Spearman: 0.8665
2021-12-08 17:29:31 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_17-16-47


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 17:35:23 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2021-12-08 17:35:27 - Cosine-Similarity :	Pearson: 0.8881	Spearman: 0.8868
2021-12-08 17:35:27 - Manhattan-Distance:	Pearson: 0.8877	Spearman: 0.8828
2021-12-08 17:35:27 - Euclidean-Distance:	Pearson: 0.8891	Spearman: 0.8841
2021-12-08 17:35:27 - Dot-Product-Similarity:	Pearson: 0.8776	Spearman: 0.8716
2021-12-08 17:35:27 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_17-16-47


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-12-08 17:41:20 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2021-12-08 17:41:23 - Cosine-Similarity :	Pearson: 0.8890	Spearman: 0.8875
2021-12-08 17:41:24 - Manhattan-Distance:	Pearson: 0.8885	Spearman: 0.8844
2021-12-08 17:41:24 - Euclidean-Distance:	Pearson: 0.8896	Spearman: 0.8846
2021-12-08 17:41:24 - Dot-Product-Similarity:	Pearson: 0.8786	Spearman: 0.8737
2021-12-08 17:41:24 - Save model to output/training_klue_sts_klue-roberta-base-2021-12-08_17-16-47


In [24]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

2021-12-08 17:47:26 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-base-2021-12-08_17-16-47
2021-12-08 17:47:27 - Use pytorch device: cuda


In [25]:
test_evaluator(model, output_path=model_save_path)

2021-12-08 17:47:29 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2021-12-08 17:47:39 - Cosine-Similarity :	Pearson: 0.7659	Spearman: 0.7590
2021-12-08 17:47:39 - Manhattan-Distance:	Pearson: 0.7654	Spearman: 0.7641
2021-12-08 17:47:39 - Euclidean-Distance:	Pearson: 0.7642	Spearman: 0.7630
2021-12-08 17:47:39 - Dot-Product-Similarity:	Pearson: 0.7366	Spearman: 0.7280


0.7641319383430123

## Kobart 요약문과 사용자 요약문 텍스트 유사도 비교




In [26]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from pandas import DataFrame

In [27]:
data = pd.read_csv('./test2.csv', encoding='CP949')
data

,원문,Kobart
0,최근 들어 도시의 경쟁력 향상을 위한 새로운 전략의 하나로 창조 도시에 대한 논의가...,최근 들어 도시의 경쟁력 향상을 위한 새로운 전략의 하나로 창조 도시에 대한 논의가...
1,창조 도시의 주된 동력을 창조 산업으로 볼 것인가 창조 계층으로 볼 것인가에 대해서...,"창조 산업을 중시하는 관점에서는, 창조 산업이 도시에 인적·사회적·문화적·경제적 다..."
2,"창조 계층을 중시하는 관점에서는, 개인의 창의력으로 부가가치를 창출하는 창조 계층이...",개인의 창의력으로 부가가치를 창출하는 창조 계층이 모여서 인재 네트워크인 창조 자본...
3,"창조성의 근본 동력을 무엇으로 보든, 한 도시가 창조 도시로 성장하려면 창조 산업과...",한 도시가 창조 도시로 성장하려면 창조 산업과 창조 계층을 유인하는 창조 환경이 먼...
4,창조 도시는 하루아침에 인위적으로 만들어지지 않으며 추진 과정에서 위험이 수반되기도...,하루아침에 인위적으로 만들어지지 않으며 추진 과정에서 위험이 수반되기도 하는 창조 ...


In [28]:
df = pd.DataFrame(data)
pd.options.display.float_format = '{:.6f}'.format

In [29]:
b=[]
c=[]

for i in range (len(df.index)):
  a = data.iloc[i]
  print("< {:d}문단 >\n".format(i+1))
  for line in  a[0].split('. '):
    print(line + '.')
  b.append(input("\n요약문을 입력하세요: "))

  embedding1 = model.encode(a[0], convert_to_tensor=True)
  embedding2 = model.encode(b[i], convert_to_tensor=True)
  cosine_score1 = util.pytorch_cos_sim(embedding1, embedding2)
  c.append(float(cosine_score1[0][0]))
  
mean = sum(c)/len(c)
print("\nOO님의 지문 이해도는 {:.1f}%입니다.".format(mean * 100))

< 1문단 >

최근 들어 도시의 경쟁력 향상을 위한 새로운 전략의 하나로 창조 도시에 대한 논의가 활발하게 진행되고 있다.
창조 도시는 창조적 인재들이 창의성을 발휘할 수 있는 환경을 갖춘 도시이다.
즉 창조 도시는 인재들을 위한 문화 및 거주 환경의 창조성이 풍부하며, 혁신적이고도 유연한 경제 시스템을 구비하고 있는 도시인 것이다..

요약문을 입력하세요: 창조 도시는 창조적 인재들이 창의성을 발휘할 수 있는 환경을 갖춘 도시이다.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

< 2문단 >

창조 도시의 주된 동력을 창조 산업으로 볼 것인가 창조 계층으로 볼 것인가에 대해서는 견해가 다소 엇갈리고 있다.
창조 산업을 중시하는 관점에서는, 창조 산업이 도시에 인적·사회적·문화적·경제적 다양성을 불어넣음으로써 도시의 재구조화를 가져오고 나아가 부가가치와 고용을 창출한다고 주장한다.
창의적 기술과 재능을 소득과 고용의 원천으로 삼는 창조 산업의 예로는 광고, 디자인, 출판, 공연 예술, 컴퓨터 게임 등이 있다..

요약문을 입력하세요: 창조 산업을 중시하는 관점에서는, 창조 산업이 도시에 인적·사회적·문화적·경제적 다양성을 불어넣음으로써 도시의 재구조화를 가져오고 나아가 부가가치와 고용을 창출한다고 주장한다.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

< 5문단 >

창조 도시는 하루아침에 인위적으로 만들어지지 않으며 추진 과정에서 위험이 수반되기도 한다.
창조 산업의 산출물은 그것에 대한 소비자의 수요와 가치 평가를 예측하기 어렵다.
또한 창조 계층의 창의력은 표준화되기 어렵고 그들의 전문화된 노동력은 대체하기가 쉽지 않다.
따라서 창조 도시를 만들기 위해서는 도시 고유의 특성을 면밀히 고찰하여 창조 산업, 창조 계층, 창조 환경의 역동성을 최대화할 수 있는 조건이 무엇인지 밝혀낼 필요가 있다..

요약문을 입력하세요: 창조 도시를 만들기 위해서는 도시 고유의 특성을 면밀히 고찰하여 창조 산업, 창조 계층, 창조 환경의 역동성을 최대화할 수 있는 조건이 무엇인지 밝혀낼 필요가 있다.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


OO님의 지문 이해도는 78.7%입니다.
